In [1]:
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_discrete_white_noise
from numpy.random import randn
import numpy as np
import sympy as sp
import math
import matplotlib.pyplot as plt
%matplotlib qt
DEG_TO_RAD = math.pi/180
RAD_TO_DEG = 180/math.pi
gravitation = 9.81
l = 0.5

In [2]:
def polar_to_kartesian(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = -r * np.cos(theta)
    return [x,y,z]

X ist der Zustand $ \vec x = \begin{bmatrix} \theta \\ \phi \\ \theta' \\ \phi' \end{bmatrix} $ wobei gilt $ q = \begin{bmatrix} \theta \\ \phi \end{bmatrix} $


Die Funktion **transfer_function** berechnet $\dot{ \vec x}$

In [3]:
def state_first_deriv(x):
    dq = x[2:]
    dq2 = second_deriv_theta_phi(x)
    
    return np.concatenate([dq, dq2])

In [4]:
def second_deriv_theta_phi(x):
    theta, phi, d_theta, d_phi = x
    c, s, t = np.cos(theta), np.sin(theta), np.tan(theta)

    d2_theta = (d_phi**2 * c - gravitation / l) * s
    d2_phi = -2 * d_theta * d_phi / t
    if (t == 0):
        print ('t is 0')

    return np.array([d2_theta, d2_phi])
    

In [5]:
def f(x, dt):
    x_new = x + state_first_deriv(x) * dt
    return x_new
    

In [6]:
def h(x):
    y = polar_to_kartesian(l, x[0], x[1]) 
    return y

In [18]:
# Nice divisors are divisors (2.5,4.2), (4, 6), (5, 2) and (1.1, inf)

x = [[np.pi / 2.5, 0, 0, np.pi/4.2]]
positions = []
Ts = [0]
dt = 1/100000
#dt = 1/300
N = 100000
for i in range(N):
    positions.append(polar_to_kartesian(l, x[i][0],x[i][1]))
    x.append(f(x[i],dt))
    Ts.append(i*dt)

In [23]:
np.pi/6 * RAD_TO_DEG

29.999999999999996

## Auswahl weniger Punkte

In [17]:
positions_decreased = []
Ts_decreased = []
for i in range(N):
    if(i % 333 == 0):
        positions_decreased.append(positions[i])
        Ts_decreased.append(Ts[i])
        

# Definition Kalman-Filter

In [18]:
sigmas = MerweScaledSigmaPoints(4, alpha=.1, beta=2, kappa=-1)
std_x, std_y, std_z = .02,.02,.02

## Simulationsdaten mit künstlichem Messrauschen versehen

In [19]:
positions_decreased = np.array(positions_decreased)


In [20]:
np.random.seed(1234)

for i in range(len(positions_decreased)):
    positions_decreased[i][0] = positions_decreased[i][0] + std_x*randn()
    positions_decreased[i][1] = positions_decreased[i][1] + std_y*randn()
    positions_decreased[i][2] = positions_decreased[i][2] + std_z*randn()
    


In [21]:
ukf = UKF(dim_x=4, dim_z=3,fx=f, hx=h, dt=dt, points=sigmas)
ukf.x = np.array(x[0])
ukf.R = np.diag([std_x**2, std_y**2, std_z**2])
ukf.Q[0:2, 0:2] = Q_discrete_white_noise(2,dt=dt, var=.02)
ukf.Q[2:4, 2:4] = Q_discrete_white_noise(2,dt=dt, var=.02)

uxs = []
for p in positions:
    ukf.predict()
    ukf.update(p)
    uxs.append(ukf.x.copy())

uxs = np.array(uxs)
ux_positions = []
for ux in uxs:
    ux_positions.append(polar_to_kartesian(l, ux[0], ux[1]))
    

In [22]:
ux_positions = np.array(ux_positions)

## Plot 

In [23]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import matplotlib as mpl

In [24]:
#mpl.rcParams['agg.path.chunksize'] = 10000

Plot Unanimated

In [25]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.set_xlim3d(-1, 1)
ax.set_ylim3d(-1, 1)
ax.set_zlim3d(-1, 1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

string = plt.quiver(0,0,0,0,0,0)

# plotting the overall curve
#ax.scatter(positions[:,0], positions[:,1], positions[:,2], c='b', marker='.', linewidth=0.00001)

def func(num, positions_decreased, pendulum): 
    x = ux_positions[num, 0]
    y = ux_positions[num, 1]
    z = ux_positions[num, 2]
    
    pendulum.set_data(x,y)
    pendulum.set_3d_properties(z)
    
    
    u = ux_positions[num, 0]
    v = ux_positions[num, 1]
    w = ux_positions[num, 2]
    
    global string
    string.remove()
    string = plt.quiver(0,0,0, u, v, w,color='g')
    
pendulum, = plt.plot([ux_positions[0,0]], [ux_positions[0,1]], [ux_positions[0,2]], c='r', marker='o')

ani = animation.FuncAnimation(fig, func, fargs=(ux_positions, pendulum), interval=1/1000, blit=False)
ax.plot(positions_decreased[:,0], positions_decreased[:,1], positions_decreased[:,2], linewidth=0.6)

#ax.plot(positions_decreased[:,0], positions_decreased[:,1], positions_decreased[:,2], linewidth=0.6)
ax.plot(ux_positions[:,0], ux_positions[:,1], ux_positions[:,2], color='red', linewidth=0.6)

### Plot every plane



In [165]:
%matplotlib inline

Plot X-Y Plane

In [65]:
plt.plot(ux_positions[:,0],ux_positions[:,1], color='red')
plt.plot(positions_decreased[:,0],positions_decreased[:,1], alpha=0.5)



Plot Y-Z Plane

In [170]:
plt.plot(ux_positions[:,1],ux_positions[:,2], color='red')
plt.plot(positions_decreased[:,1],positions_decreased[:,2], alpha=0.5)


Plot X-Z Plane

In [171]:
plt.plot(ux_positions[:,0],ux_positions[:,2], color='red')
plt.plot(positions_decreased[:,0],positions_decreased[:,2], alpha=0.5)
